<a href="https://www.kaggle.com/code/marcellemmer/context-creation?scriptVersionId=153407154" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# Context creation using Wikipedia

We downloaded the 13GB Wikipedia Plaintext (2023-07-01) dataset from Kaggle. This dataset contains all the wikipedia articles up to the mentioned date stored in parquet files. We use only the wiki_2023_index.parquet file that contains the first sentences of the articles as context for our model. We assing a context column for each question of the Q&A dataframe from this file. This is done with the Sentence Transformer library that embeds the wikipedia articles and with Faiss that does a similarity search between the question and the first sentences of the articles. We retrieve the most similar wikipedia article for each question.

## Sources

* https://www.kaggle.com/datasets/jjinho/wikipedia-20230701/data?select=h.parquet

* https://github.com/facebookresearch/faiss/wiki

In [1]:
!pip install datasets
!pip install faiss-gpu sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 13.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 8.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - \ done
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125923 sha256=20c9657b8cddd0fb1c4ec903216a974bfc5a004e9602c907feab06cb65b436f1
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence-transformers


In [2]:
# Importing the libraries
import os
import pandas as pd
from datasets import load_dataset
import faiss
from sentence_transformers import SentenceTransformer
import torch

# Important parameters describing the code
SIM_MODEL = 'all-MiniLM-L6-v2'
# Set device
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'

#Loading the questions
qna_df = pd.read_csv("https://raw.githubusercontent.com/csabi0312/DeepLProject/main/train.csv",index_col=0)

qna_df.head()

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


,prompt,A,B,C,D,E,answer
id,,,,,,,
0,Which of the following statements accurately d...,MOND is a theory that reduces the observed mis...,MOND is a theory that increases the discrepanc...,MOND is a theory that explains the missing bar...,MOND is a theory that reduces the discrepancy ...,MOND is a theory that eliminates the observed ...,D
1,Which of the following is an accurate definiti...,Dynamic scaling refers to the evolution of sel...,Dynamic scaling refers to the non-evolution of...,Dynamic scaling refers to the evolution of sel...,Dynamic scaling refers to the non-evolution of...,Dynamic scaling refers to the evolution of sel...,A
2,Which of the following statements accurately d...,The triskeles symbol was reconstructed as a fe...,The triskeles symbol is a representation of th...,The triskeles symbol is a representation of a ...,The triskeles symbol represents three interloc...,The triskeles symbol is a representation of th...,A
3,What is the significance of regularization in ...,Regularizing the mass-energy of an electron wi...,Regularizing the mass-energy of an electron wi...,Regularizing the mass-energy of an electron wi...,Regularizing the mass-energy of an electron wi...,Regularizing the mass-energy of an electron wi...,C
4,Which of the following statements accurately d...,The angular spacing of features in the diffrac...,The angular spacing of features in the diffrac...,The angular spacing of features in the diffrac...,The angular spacing of features in the diffrac...,The angular spacing of features in the diffrac...,D


In [3]:
# Load Parquet files into a Hugging Face dataset
# Source: https://www.kaggle.com/datasets/jjinho/wikipedia-20230701/data?select=wiki_2023_index.parquet
wiki_dataset = load_dataset('parquet', data_files={'train': "/kaggle/input/wikipedia-20230701/wiki_2023_index.parquet"}, split='train') # 1.76GB file

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Dataset parquet downloaded and prepared to /root/.cache/huggingface/datasets/parquet/default-970463e2dbb8e6aa/0.0.0/0b6d5799bb726b24ad7fc7be720c170d8e497f575d02d47537de9a5bac074901. Subsequent calls will reuse this data.


In [4]:
# Load pre-trained sentence transformer model
model = SentenceTransformer(SIM_MODEL)

# Create a Faiss index
index = faiss.IndexFlatIP(model.get_sentence_embedding_dimension())

# Define batch size
batch_size = 500_000

# Iterate over the dataset in batches
for i in range(0, len(wiki_dataset['context']), batch_size):
    # Encode the context sentences using the SentenceTransformer model
    context_embeddings = model.encode(wiki_dataset['context'][i:i+batch_size],
                                      device=DEVICE,
                                      show_progress_bar=True,
                                      convert_to_tensor=True,
                                      normalize_embeddings=True).half()  # Use mixed-precision training (FP16) to reduce memory footprint

    # Convert the embeddings to a numpy array
    context_embeddings_np = context_embeddings.detach().cpu().numpy()
    context_embeddings_np = context_embeddings_np.astype('float32')

    # Add the embeddings to the Faiss index
    index.add(context_embeddings_np)

    # Free up memory
    del context_embeddings, context_embeddings_np

# Function to retrieve most similar documents
def retrieve_most_similar(query, k=20):
    query_embedding = model.encode(query, device=DEVICE, show_progress_bar=True, convert_to_tensor=True, normalize_embeddings=True)
    query_embedding = query_embedding.reshape(1, -1)  # Reshape for Faiss
    query_embedding = query_embedding.detach().cpu().numpy()
    _, idx = index.search(query_embedding, k)
    return idx[0]

# Example usage
query_text = qna_df['prompt'][0]
print(f'example prompt {query_text}')
similar_documents_indices = retrieve_most_similar(query_text)

# Print similar documents
for idx in similar_documents_indices:
    print(wiki_dataset[int(idx)]['context'])

Batches:   0%|          | 0/15625 [00:00<?, ?it/s]

Batches:   0%|          | 0/15625 [00:00<?, ?it/s]

Batches:   0%|          | 0/15625 [00:00<?, ?it/s]

Batches:   0%|          | 0/15625 [00:00<?, ?it/s]

Batches:   0%|          | 0/15625 [00:00<?, ?it/s]

...

Batches:   0%|          | 0/15625 [00:00<?, ?it/s]

Batches:   0%|          | 0/8962 [00:00<?, ?it/s]

example prompt Which of the following statements accurately describes the impact of Modified Newtonian Dynamics (MOND) on the observed "missing baryonic mass" discrepancy in galaxy clusters?


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

In cosmology, the missing baryon problem is an observed discrepancy between the amount of baryonic matter detected from shortly after the Big Bang and from more recent epoch
A mass deficit is the amount of mass (in stars) that has been removed from the center of a galaxy, presumably by the action of a binary supermassive black hol
MACS J0416.1-2403 is a cluster of galaxies at a redshift of z=0.397 with a mass 160 trillion times the mass of the Sun inside . Its mass extends out to a radius of and was measured as 1.15 × 1015 solar masse
The dwarf galaxy problem, also known as the missing satellites problem, arises from a mismatch between observed dwarf galaxy numbers and collisionless numerical cosmological simulations that predict the evolution of the distribution of matter in the univers
Galaxy groups and clusters are the largest known gravitationally bound objects to have arisen thus far in the process of cosmic structure formatio
In astronomy, a RAMBO or robust association of massive

In [5]:
# Create the context column from the wikipedia article
# Create an empty list to store the context for each prompt
context_list = []

# Loop through each prompt in the qna_df dataframe
for i in range(len(qna_df)):
    query_text = qna_df['prompt'][i]
    similar_documents_indices = retrieve_most_similar(query_text)

    # Get the first answer from the corresponding wiki_dataset
    context = wiki_dataset[int(similar_documents_indices[0])]['context']


    context_list.append(context)

# Add the context_list as a new column "context" to the qna_df DataFrame
qna_df['context'] = context_list

# Save the Q&A DataFrame to a CSV file
qna_df.to_csv('openbook-qna-data.csv', index=False)

# Display the modified DataFrame
qna_df.head()

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

...

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

,prompt,A,B,C,D,E,answer,context
id,,,,,,,,
0,Which of the following statements accurately d...,MOND is a theory that reduces the observed mis...,MOND is a theory that increases the discrepanc...,MOND is a theory that explains the missing bar...,MOND is a theory that reduces the discrepancy ...,MOND is a theory that eliminates the observed ...,D,"In cosmology, the missing baryon problem is an..."
1,Which of the following is an accurate definiti...,Dynamic scaling refers to the evolution of sel...,Dynamic scaling refers to the non-evolution of...,Dynamic scaling refers to the evolution of sel...,Dynamic scaling refers to the non-evolution of...,Dynamic scaling refers to the evolution of sel...,A,Dynamic scaling (sometimes known as Family-Vic...
2,Which of the following statements accurately d...,The triskeles symbol was reconstructed as a fe...,The triskeles symbol is a representation of th...,The triskeles symbol is a representation of a ...,The triskeles symbol represents three interloc...,The triskeles symbol is a representation of th...,A,thumb|Neolithic triple spiral symbol A triskel...
3,What is the significance of regularization in ...,Regularizing the mass-energy of an electron wi...,Regularizing the mass-energy of an electron wi...,Regularizing the mass-energy of an electron wi...,Regularizing the mass-energy of an electron wi...,Regularizing the mass-energy of an electron wi...,C,"In physics, especially quantum field theory, r..."
4,Which of the following statements accurately d...,The angular spacing of features in the diffrac...,The angular spacing of features in the diffrac...,The angular spacing of features in the diffrac...,The angular spacing of features in the diffrac...,The angular spacing of features in the diffrac...,D,Kinematic diffraction is the approach to study...
